In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#float64のカラム3つを削除し、ランダムフォレストオブジェトで重要度の低い特徴量を削減。
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

#学習データ・テストデータの読み込み
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
#学習データを特徴量と目的変数に分ける
train_x = train.drop(['Id','SalePrice','LotFrontage','MasVnrArea','GarageYrBlt'], axis=1)
train_y = train['SalePrice']
#テストデータ
test_x = test.drop(['Id','LotFrontage','MasVnrArea','GarageYrBlt'], axis=1)
test_y_answer = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv').drop(['Id'], axis=1)

#それぞれのカテゴリ変数にlabel encodingを適用する
x = pd.concat([train_x, test_x], axis=0)
cat_cols = x.dtypes[train_x.dtypes=='object'].index.tolist()
for c in cat_cols:
#学習データに基づいてどう変換するかを定める
    le = LabelEncoder()
    le.fit(x[c].fillna('NaN'))
    
    #学習データ・テストデータを変換する
    train_x[c] = le.transform(train_x[c].fillna('NaN'))
    test_x[c] = le.transform(test_x[c].fillna('NaN'))

#ランダムフォレストオブジェトの生成(決定木の個数=500)
forest = RandomForestClassifier(n_estimators=500, random_state=1)
#モデルを適合
forest.fit(train_x, train_y)
#特徴量の重要度を抽出
importances = forest.feature_importances_
#重要度の降順で特徴量のインデックスを抽出
indices = np.argsort(importances)[::-1]
#重要度の降順で特徴量の名称、重要度を表示
for f in range(train_x.shape[1]):
    print("%2d) %-*s %f" %
         (f + 1, 30, train_x.columns[indices[f]], importances[indices[f]])
         )

In [ ]:
#bestなモデルに編集していく
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


train_x_2 = train_x[train_x.columns[indices[np.arange(27)]]]
test_x_2 = test_x[test_x.columns[indices[np.arange(27)]]]
#バリデーション
tr_x, va_x, tr_y, va_y = train_test_split(train_x_2, train_y,
                                         test_size=0.25, random_state=71, shuffle=True)

#特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x_2)

#ハイパーパラメータの設定
params = {'objective': 'reg:squarederror', 'random_state': 71}
num_round=50
#学習の実行
#バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
#watchlistには学習データおよびバリデーションデータをセットする
watchlist = [(dtrain, 'train'), (dvalid, 'eval') ]
model = xgb.train(params, dtrain, num_round, early_stopping_rounds=20, evals=watchlist)

va_pred = model.predict(dvalid)
print('バリデーションデータ予測値:')
print(va_pred)
#平均平方二乗誤差でスコアを算出
score = np.sqrt(mean_squared_error(va_y, va_pred))
print('平均平方二乗誤差')
print(score)

#予測(二値の予測値)
test_pred = model.predict(dtest)
print('test予測値:')
print(test_pred)
#平均平方二乗誤差でスコアを算出
score_test = np.sqrt(mean_squared_error(test_y_answer, test_pred))
print('test平均平方二乗誤差')
print(score_test)

In [ ]:
data1 = {
    'Id'       : test['Id'],
    'Saleprice': test_pred
}
df1 = pd.DataFrame(data_test)
print(df1)

df1.to_csv("test1.csv", index = False)

In [ ]:
#全特徴量を重要順にどこまで使用するかを確認する
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
score_list = []
score_va_list = []
score_test_list = []
kf = KFold(n_splits=4, shuffle=True, random_state=71)

#Salepriceと関連性の高い順に特徴量を1~30個使用
for p in range(1,80):
    train_x_2 = train_x[train_x.columns[indices[np.arange(p)]]]
    test_x_2 = test_x[test_x.columns[indices[np.arange(p)]]]
#バリデーション
    for tr_idx, va_idx in kf.split(train_x_2):
        tr_x, va_x = train_x_2.iloc[tr_idx], train_x_2.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

#特徴量と目的変数をxgboostのデータ構造に変換する
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        dtest = xgb.DMatrix(test_x_2)

#ハイパーパラメータの設定
        params = {
            'booster':'gbtree',
            'objective': 'reg:squarederror',
            'eta':0.1,
            'gamma':0.0,
            'alpha':0.0,
            'lambda':1.0,
            'min_child_weight':1,
            'max_depth':5,
            'subsample':0.8,
            'colsample_bytree':0.8,
            'random_state': 71
        }
        num_round=100

#学習の実行
#バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
#watchlistには学習データおよびバリデーションデータをセットする
        watchlist = [(dtrain, 'train'), (dvalid, 'eval') ]
        model = xgb.train(params, dtrain, num_round)

        va_pred = model.predict(dvalid)
#平均平方二乗誤差でスコアを算出
        score = np.sqrt(mean_squared_error(va_y, va_pred))
        score_list.append(score)
        continue
        
    score_va_list.append(np.mean(score_list))
    
#test予測(二値の予測値)
    test_pred = model.predict(dtest)
#平均平方二乗誤差でtestスコアを算出
    score_test = np.sqrt(mean_squared_error(test_y_answer, test_pred))
    score_test_list.append(score_test)
    
print('平均平方二乗誤差スコア')    
for f in range(len(score_va_list)):
    print("%2d) バリデーションのスコア：%-*s 追加した特徴量は %-*s|　testのスコア：%5s" 
          %(f + 1, 
            20, score_va_list[f],
            30, train_x.columns[indices[f]],
            score_test_list[f]
           )
          )

In [ ]:
#ここから読み込む
#float64のカラムのNaNを0に置換し、ランダムフォレストオブジェトで重要の高い順に特徴量を表示。
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

#学習データ・テストデータの読み込み
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
#学習データを特徴量と目的変数に分ける
train_x = train.drop(['Id','SalePrice'], axis=1)
train_y = train['SalePrice']
#テストデータ
test_x = test.drop(['Id'], axis=1)
test_y_answer = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv').drop(['Id'], axis=1)


#それぞれのカテゴリ変数にlabel encodingを適用する
x = pd.concat([train_x, test_x], axis=0)
cat_cols = x.dtypes[train_x.dtypes=='object'].index.tolist()
#floatの特徴量をintに変換する
train_x[['LotFrontage','MasVnrArea','GarageYrBlt']] = train_x[['LotFrontage','MasVnrArea','GarageYrBlt']].fillna(0).astype(int)
test_x[['LotFrontage','MasVnrArea','GarageYrBlt']] =test_x[['LotFrontage','MasVnrArea','GarageYrBlt']].fillna(0).astype(int)
for c in cat_cols:
#学習データに基づいてどう変換するかを定める
    le = LabelEncoder()
    le.fit(x[c].fillna('NaN'))
    #学習データ・テストデータを変換する
    train_x[c] = le.transform(train_x[c].fillna('NaN'))
    test_x[c] = le.transform(test_x[c].fillna('NaN'))

#ランダムフォレストオブジェトの生成(決定木の個数=500)
forest = RandomForestClassifier(n_estimators=500, random_state=1)
#モデルを適合
forest.fit(train_x, train_y)
#特徴量の重要度を抽出
importances = forest.feature_importances_
#重要度の降順で特徴量のインデックスを抽出
indices = np.argsort(importances)[::-1]
#重要度の降順で特徴量の名称、重要度を表示
for f in range(train_x.shape[1]):
    print("%2d) %-*s %f" %
         (f + 1, 30, train_x.columns[indices[f]], importances[indices[f]])
         )

In [ ]:
#予測をcsvで保存して、試しに提出する
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
kf = KFold(n_splits=4, shuffle=True, random_state=71)

#ハイパーパラメータの設定
score_list = []
params = {
        'booster':'gbtree',
        'objective': 'reg:squarederror',
        'eta':0.01,
        'gamma':0.0,
        'alpha':0.0,
        'lambda':1.0,
        'min_child_weight':2,
        'max_depth':6,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'colsample_bylevel':0.4,
        'random_state': 71
        }
num_round=1000

train_x_2 = train_x
test_x_2 = test_x

#バリデーション
for tr_idx, va_idx in kf.split(train_x_2):
    tr_x, va_x = train_x_2.iloc[tr_idx], train_x_2.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

#特徴量と目的変数をxgboostのデータ構造に変換する
    dtrain = xgb.DMatrix(tr_x, label=tr_y)
    dvalid = xgb.DMatrix(va_x, label=va_y)

#学習の実行
#バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
#watchlistには学習データおよびバリデーションデータをセットする
    watchlist = [(dtrain, 'train'), (dvalid, 'eval') ]
    model = xgb.train(params, dtrain, num_round)

    va_pred = model.predict(dvalid)
#平均平方二乗誤差でスコアを算出
    score = np.sqrt(mean_squared_error(va_y, va_pred))
    score_list.append(score)
    continue
        
print(np.mean(score_list))
    
#test予測(二値の予測値)
dtrain = xgb.DMatrix(train_x_2, label=train_y)
dtest = xgb.DMatrix(test_x_2)

model = xgb.train(params, dtrain, num_round)
test_pred = model.predict(dtest)

data2 = {
    'Id'       : test['Id'],
    'Saleprice': test_pred
}
df2 = pd.DataFrame(data2)
print(df2)

df2.to_csv("test6.csv", index = False)

In [ ]:
#手動でパラメータチューニングする場合
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
kf = KFold(n_splits=4, shuffle=True, random_state=71)

#ハイパーパラメータの設定
for i in [0.0,0.1,0.2,0.3,0.4]:
    score_list = []
    params = {
        'booster':'gbtree',
        'objective': 'reg:squarederror',
        'eta':0.01,
        'gamma':i,
        'alpha':0.0,
        'lambda':1.0,
        'min_child_weight':2,
        'max_depth':6,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'colsample_bylevel':0.4,
        'random_state': 71
        }
    num_round=1000

    train_x_2 = train_x
    test_x_2 = test_x

#バリデーション
    for tr_idx, va_idx in kf.split(train_x_2):
        tr_x, va_x = train_x_2.iloc[tr_idx], train_x_2.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

#特徴量と目的変数をxgboostのデータ構造に変換する
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)

#学習の実行
#バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
#watchlistには学習データおよびバリデーションデータをセットする
        watchlist = [(dtrain, 'train'), (dvalid, 'eval') ]
        model = xgb.train(params, dtrain, num_round)

        va_pred = model.predict(dvalid)
#平均平方二乗誤差でスコアを算出
        score = np.sqrt(mean_squared_error(va_y, va_pred))
        score_list.append(score)
        continue
        
    print("i = %s" %(i))    
    print(np.mean(score_list))
    
#test予測(二値の予測値)
    dtrain = xgb.DMatrix(train_x_2, label=train_y)
    dtest = xgb.DMatrix(test_x_2)

    model = xgb.train(params, dtrain, num_round)
    test_pred = model.predict(dtest)

    data2 = {
        'Id'       : test['Id'],
        'Saleprice': test_pred
    }
    df2 = pd.DataFrame(data2)
    print(df2)

    df2.to_csv("test%s.csv" %(i), index = False)

In [ ]:
#グリッドサーチでパラメータチューニング
import itertools
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
kf = KFold(n_splits=4, shuffle=True, random_state=71)

#ハイパーパラメータの設定
param_space = {
    'max_depth':[2,3,4,5,6,7,8,9,10],
    'min_child_weight':[1,2,3,4],
    }
param_combinations = itertools.product(param_space['max_depth'],param_space['min_child_weight'])
parameters = []
scores = []
score_list = []
num_round=1000
train_x_2 = train_x
test_x_2 = test_x

for  max_depth, min_child_weight in param_combinations:
    params = {
        'booster':'gbtree',
        'objective': 'reg:squarederror',
        'eta':0.01,
        'gamma':0.0,
        'alpha':0.0,
        'lambda':1.0,
        'max_depth':max_depth,
        'min_child_weight':min_child_weight,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'colsample_bylevel':0.4,
        'random_state': 71
        }

#バリデーション
    for tr_idx, va_idx in kf.split(train_x_2):
        tr_x, va_x = train_x_2.iloc[tr_idx], train_x_2.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

#特徴量と目的変数をxgboostのデータ構造に変換する
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)

#学習の実行
#バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
#watchlistには学習データおよびバリデーションデータをセットする
        watchlist = [(dtrain, 'train'), (dvalid, 'eval') ]
        model = xgb.train(params, dtrain, num_round)

        va_pred = model.predict(dvalid)
#平均平方二乗誤差でスコアを算出
        score = np.sqrt(mean_squared_error(va_y, va_pred))
        score_list.append(score)
        continue
    print(f'max_depth: %s, min_chind_weight: %s'%(max_depth, min_child_weight))
    print(np.mean(score_list))
    
    parameters.append((max_depth,min_child_weight))
    scores.append(np.mean(score_list))

best_idx = np.argsort(scores)[0]
best_param = parameters[best_idx]
print(f'max_depth: {best_param[0]}, min_chind_weight: {best_param[1]}')